In [1]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

In [20]:
census_bg_df = gpd.read_file('../data/ma_tracts_census_data.geojson')
census_bg_df = census_bg_df.to_crs(crs="EPSG:26986")
stops_with_buffer = gpd.read_file('../data/mbta_stops.geojson')
stops_with_buffer = stops_with_buffer.to_crs(crs="EPSG:26986")

In [59]:
# Compute area of census bg
census_bg_df['bg_area_km2'] = census_bg_df.area / 1e6

In [21]:
mbtac_gdf = gpd.read_file("../data/preprocessed_data/mbta_municipalities.geojson")
mbtac_gdf = mbtac_gdf.to_crs(crs="EPSG:26986")

In [22]:
# m = census_bg_df.explore()
# m = stops_with_buffer.explore(m=m, color='#FF00001A')
# 
# m

In [23]:
census_bg_df.head(10)

,STATEFP,COUNTYFP,Geo_FIPS,GEOID,NAME,Geo_QNAME,MTFCC,ALAND,AWATER,INTPTLAT,...,Occupied Housing Units: 3 Vehicles Available,Occupied Housing Units: 4 Vehicles Available,Occupied Housing Units: 5 or More Vehicles Available,% Occupied Housing Units: No Vehicle Available,% Occupied Housing Units: 1 Vehicle Available,% Occupied Housing Units: 2 Vehicles Available,% Occupied Housing Units: 3 Vehicles Available,% Occupied Housing Units: 4 Vehicles Available,% Occupied Housing Units: 5 or More Vehicles Available,geometry
0,25,001,010100,25001010100,101,Census Tract 101,G5020,25046217,12765872,+42.0598291,...,138,8,0,12.27,56.56,23.85,6.91,0.4,0,"MULTIPOLYGON (((307408.166 865567.332, 307404...."
1,25,001,010206,25001010206,102.06,Census Tract 102.06,G5020,51240917,18830100,+41.9226356,...,240,65,14,7.65,33.06,43.14,12.15,3.29,0.71,"MULTIPOLYGON (((327055.293 848714.028, 326587...."
2,25,001,010208,25001010208,102.08,Census Tract 102.08,G5020,54268861,11461462,+42.0135566,...,134,0,23,4.66,33.85,46.27,13,0,2.23,"POLYGON ((316236.176 865726.108, 316165.574 86..."
3,25,001,010304,25001010304,103.04,Census Tract 103.04,G5020,18347661,7830614,+41.8251080,...,157,43,58,0.88,38.81,41.38,11.52,3.15,4.26,"MULTIPOLYGON (((324284.467 839991.724, 324281...."
4,25,001,010306,25001010306,103.06,Census Tract 103.06,G5020,17828555,1730602,+41.8593758,...,188,34,39,2.13,42.32,36.37,13.81,2.5,2.87,"POLYGON ((327782.186 844560.250, 327755.093 84..."
5,25,001,010400,25001010400,104,Census Tract 104,G5020,15587077,8476393,+41.7541457,...,241,53,0,6.93,40.39,34.15,15.19,3.34,0,"MULTIPOLYGON (((325728.645 840191.270, 325736...."
6,25,001,010500,25001010500,105,Census Tract 105,G5020,21096183,13691016,+41.7662035,...,269,55,42,3.18,24.56,46.44,18.98,3.88,2.96,"MULTIPOLYGON (((328572.331 834684.747, 328594...."
7,25,001,010600,25001010600,106,Census Tract 106,G5020,14695400,14524028,+41.6995108,...,165,10,0,5.66,36.25,47.07,10.38,0.63,0,"MULTIPOLYGON (((329840.329 824235.404, 329835...."
8,25,001,010700,25001010700,107,Census Tract 107,G5020,27186448,29106036,+41.6185172,...,220,61,12,3.12,35.18,44.47,12.94,3.59,0.71,"MULTIPOLYGON (((327513.821 822860.123, 327530...."
9,25,001,010800,25001010800,108,Census Tract 108,G5020,28389104,3127182,+41.7434889,...,242,171,72,6.27,31.24,41.09,10.68,7.55,3.18,"POLYGON ((318266.528 836686.440, 318280.268 83..."


In [24]:
assert census_bg_df.crs == stops_with_buffer.crs

In [241]:
columns_to_weight = [
    'Total Population',
    'Population Density (Per Sq. Mile)',
    'Total Population:',
    'Total Population: Male',
    'Total Population: Female',
    '% Total Population: Male',
    '% Total Population: Female',
    'Total Population.1',
    'Total Population: Not Hispanic or Latino',
    'Total Population: Not Hispanic or Latino: White Alone',
    'Total Population: Not Hispanic or Latino: Black or African American Alone',
    'Total Population: Not Hispanic or Latino: American Indian and Alaska Native Alone',
    'Total Population: Not Hispanic or Latino: Asian Alone',
    'Total Population: Not Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone',
    'Total Population: Not Hispanic or Latino: Some Other Race Alone',
    'Total Population: Not Hispanic or Latino: Two or More Races',
    'Total Population: Hispanic or Latino',
    'Total Population: Hispanic or Latino: White Alone',
    'Total Population: Hispanic or Latino: Black or African American Alone',
    'Total Population: Hispanic or Latino: American Indian and Alaska Native Alone',
    'Total Population: Hispanic or Latino: Asian Alone',
    'Total Population: Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone',
    'Total Population: Hispanic or Latino: Some Other Race Alone',
    'Total Population: Hispanic or Latino: Two or More Races',
    '% Total Population: Not Hispanic or Latino',
    '% Total Population: Not Hispanic or Latino: White Alone',
    '% Total Population: Not Hispanic or Latino: Black or African American Alone',
    '% Total Population: Not Hispanic or Latino: American Indian and Alaska Native Alone',
    '% Total Population: Not Hispanic or Latino: Asian Alone',
    '% Total Population: Not Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone',
    '% Total Population: Not Hispanic or Latino: Some Other Race Alone',
    '% Total Population: Not Hispanic or Latino: Two or More Races',
    '% Total Population: Hispanic or Latino',
    '% Total Population: Hispanic or Latino: White Alone',
    '% Total Population: Hispanic or Latino: Black or African American Alone',
    '% Total Population: Hispanic or Latino: American Indian and Alaska Native Alone',
    '% Total Population: Hispanic or Latino: Asian Alone',
    '% Total Population: Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone',
    '% Total Population: Hispanic or Latino: Some Other Race Alone',
    '% Total Population: Hispanic or Latino: Two or More Races',
    'Median Household Income (In 2022 Inflation Adjusted Dollars)',
    'Workers 16 Years and Over:',
    'Workers 16 Years and Over: Car, Truck, or Van',
    'Workers 16 Years and Over: Drove Alone',
    'Workers 16 Years and Over: Carpooled',
    'Workers 16 Years and Over: Public Transportation (Includes Taxicab)',
    'Workers 16 Years and Over: Motorcycle',
    'Workers 16 Years and Over: Bicycle',
    'Workers 16 Years and Over: Walked',
    'Workers 16 Years and Over: Other Means',
    'Workers 16 Years and Over: Worked At Home',
    '% Workers 16 Years and Over: Car, Truck, or Van',
    '% Workers 16 Years and Over: Drove Alone',
    '% Workers 16 Years and Over: Carpooled',
    '% Workers 16 Years and Over: Public Transportation (Includes Taxicab)',
    '% Workers 16 Years and Over: Motorcycle',
    '% Workers 16 Years and Over: Bicycle',
    '% Workers 16 Years and Over: Walked',
    '% Workers 16 Years and Over: Other Means',
    '% Workers 16 Years and Over: Worked At Home',
    'Occupied Housing Units',
    'Occupied Housing Units: No Vehicle Available',
    'Occupied Housing Units: 1 Vehicle Available',
    'Occupied Housing Units: 2 Vehicles Available',
    'Occupied Housing Units: 3 Vehicles Available',
    'Occupied Housing Units: 4 Vehicles Available',
    'Occupied Housing Units: 5 or More Vehicles Available',
    '% Occupied Housing Units: No Vehicle Available',
    '% Occupied Housing Units: 1 Vehicle Available',
    '% Occupied Housing Units: 2 Vehicles Available',
    '% Occupied Housing Units: 3 Vehicles Available',
    '% Occupied Housing Units: 4 Vehicles Available',
    '% Occupied Housing Units: 5 or More Vehicles Available',
]

columns_to_weight_ints = [i for i in columns_to_weight if '%' not in i]
columns_to_weight_perc = [i for i in columns_to_weight if '%' in i]
renamed_columns = ['weighted_' + str(i) for i in columns_to_weight]

In [242]:
# Compute area of census bg
census_bg_df[columns_to_weight] = census_bg_df[columns_to_weight].astype(float).fillna(0)

In [243]:
# Assume 'stops_with_buffer' and 'census_bg_df' are defined and are valid GeoDataFrames

# Initialize an empty GeoDataFrame with a specified CRS
station_buffer_census_df = gpd.GeoDataFrame()

# Loop through each station in 'stops_with_buffer'
for idx, station in tqdm(stops_with_buffer.iterrows()):
    # Create a GeoDataFrame for the current station's buffer geometry
    curr_gdf = gpd.GeoDataFrame({'geometry': [station['geometry']]}, crs=stops_with_buffer.crs)

    # Clip the census blocks to the current station's buffer
    curr_census = census_bg_df.clip(curr_gdf)

    # Project the clipped census blocks to the desired CRS
    curr_census = curr_census.to_crs(crs="EPSG:26986")

    # Calculate the area in square kilometers
    curr_census['clipped_area_km2'] = curr_census['geometry'].area / 1e6
    
    # Calculate the respective area percent
    curr_census['station_buffer_area_km2'] = station['geometry'].area / 1e6
    curr_census['percent_of_buffer_area'] = curr_census['clipped_area_km2'] / curr_census['station_buffer_area_km2']

    # Calculate the percent of the block group area
    curr_census['percent_of_census_bg_area'] = curr_census['clipped_area_km2'] / curr_census['bg_area_km2']
    
    # Recompute columns to reflect the spatial weighting with respect to the census block group
    # Columns that are in absolute values can just be multiplied by the area that is in the station buffer
    weighted_census = curr_census[columns_to_weight].copy()
    weighted_census.loc[:, columns_to_weight_ints] = weighted_census[columns_to_weight_ints].mul(curr_census['percent_of_census_bg_area'], axis=0)
    weighted_census.loc[:, columns_to_weight_ints] = weighted_census[columns_to_weight_ints].round(decimals=0).astype(int)
    # The columns that are in percent, however, have to be weighted by the percent of buffer area that is covered by that BG
    weighted_census.loc[:, columns_to_weight_perc] = curr_census[columns_to_weight_perc].mul(curr_census['percent_of_buffer_area'], axis=0)

    # Make sure that we do not have any percent that are bigger than 100% !!!
    assert (weighted_census[[i for i in weighted_census.columns if '%' in i]] > 100).any().any() == False
    
    # Finally, we assign the calculated values to the original census
    curr_census[renamed_columns] = weighted_census.rename(columns={o: n for o, n in zip(columns_to_weight, renamed_columns)})
    
    # We can now just add all the values in the curr_census and add the row to the respective station entry
    summed_census = curr_census[renamed_columns].sum(axis=0)
    stops_with_buffer.loc[stops_with_buffer.index==idx, renamed_columns] = summed_census[renamed_columns].to_numpy()

    # Add an identifier for the current station (e.g., the loop index or the station's index from 'stops_with_buffer')
    curr_census['stop_id'] = station['stop_id']

    # Append the current census blocks to the main GeoDataFrame
    station_buffer_census_df = pd.concat([station_buffer_census_df, curr_census], ignore_index=True)


7324it [05:48, 21.02it/s]


In [265]:
curr_census['stop_id']

350    WR-0329-02
348    WR-0329-02
339    WR-0329-02
349    WR-0329-02
347    WR-0329-02
Name: stop_id, dtype: object

In [244]:
brookline_stations = station_buffer_census_df.sjoin(mbtac_gdf[mbtac_gdf['municipality'] == 'Brookline'])['stop_id'].unique()
station_buffer_census_df[station_buffer_census_df['stop_id'].isin(brookline_stations)].head(5)

,STATEFP,COUNTYFP,Geo_FIPS,GEOID,NAME,Geo_QNAME,MTFCC,ALAND,AWATER,INTPTLAT,...,weighted_Occupied Housing Units: 3 Vehicles Available,weighted_Occupied Housing Units: 4 Vehicles Available,weighted_Occupied Housing Units: 5 or More Vehicles Available,weighted_% Occupied Housing Units: No Vehicle Available,weighted_% Occupied Housing Units: 1 Vehicle Available,weighted_% Occupied Housing Units: 2 Vehicles Available,weighted_% Occupied Housing Units: 3 Vehicles Available,weighted_% Occupied Housing Units: 4 Vehicles Available,weighted_% Occupied Housing Units: 5 or More Vehicles Available,stop_id
166,25,021,400600,25021400600,4006,Census Tract 4006,G5020,1226102,553,+42.3322718,...,13.0,7.0,0.0,2.423517,7.407197,2.580788,0.310674,0.168873,0.000000,1027
167,25,021,401100,25021401100,4011,Census Tract 4011,G5020,4131254,109096,+42.3238534,...,11.0,0.0,0.0,1.591667,5.008833,7.945434,1.515873,0.064505,0.000000,1027
168,25,017,373600,25017373600,3736,Census Tract 3736,G5020,3008076,92136,+42.3300314,...,0.0,0.0,0.0,0.019146,0.106757,0.136611,0.048680,0.003260,0.005178,1027
169,25,025,000502,25025000502,5.02,Census Tract 5.02,G5020,847704,334529,+42.3315048,...,0.0,4.0,0.0,18.721278,13.247782,6.303163,0.000000,0.138278,0.000000,1027
170,25,025,000503,25025000503,5.03,Census Tract 5.03,G5020,139345,0,+42.3396332,...,8.0,0.0,0.0,3.151600,2.992296,0.703421,0.048964,0.000000,0.000000,1027


In [264]:
station_buffer_census_df_complete = pd.merge(station_buffer_census_df, stops_with_buffer[[i for i in stops_with_buffer.columns if i not in renamed_columns]], how='left', on='stop_id')
station_buffer_census_df_complete['geometry'] = station_buffer_census_df_complete['geometry_x']
del station_buffer_census_df_complete['geometry_x']
del station_buffer_census_df_complete['geometry_y']
station_buffer_census_df_complete = gpd.GeoDataFrame(station_buffer_census_df_complete)

In [267]:
station_buffer_census_df_complete[station_buffer_census_df_complete['stop_id']=='WR-0329-02'].explore()

In [268]:
station_buffer_census_df_complete.to_file('../data/preprocessed_data/station_buffer_census_bgs_separate.geojson')
stops_with_buffer.to_file('../data/preprocessed_data/station_buffer_census_cumulative.geojson')

You can find the above files on GDrive:

- `station_buffer_census_bgs_separate.geojson`: [GDrive Link](https://drive.google.com/file/d/1AAlo7GSMCMSBJOBQTCzS9KA7ZcD2m9uI/view?usp=drive_link)
- `station_buffer_census_cumulative.geojson`: [GDrive Link](https://drive.google.com/file/d/181sMgDXKSQLBNk647Dw_5VApV3WfjGP1/view?usp=drive_link)